In [1]:
import os
import pathlib
import json
import io
import numpy as np
import tensorflow as tf
from random import shuffle
from shutil import rmtree
from object_detection.utils import label_map_util

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
from xml.dom import minidom

import pandas as pd

In [2]:
PROJECT_CONFIG_PATH = 'workspace/road_damage/training-config.json'

In [3]:
def load_config():
    with open(PROJECT_CONFIG_PATH, 'r') as f:
        return json.load(f)
        

In [4]:
PROJECT_CONFIG = load_config()

In [5]:
def create_name_index():
    categories_map = label_map_util.create_categories_from_labelmap((PROJECT_CONFIG['PATH_TO_LABELS']))
    name_index = {}
    for cat in categories_map:
        name_index[cat['name']] = cat
    return name_index

In [6]:
def load_from_path():
    model = tf.saved_model.load(PROJECT_CONFIG['MODEL_PATH'])
    model = model.signatures['serving_default']
    return model

In [7]:
MODEL = load_from_path()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
def xml_to_csv(annotations):
    xml_list = []
    for idx, xml_file in enumerate(annotations):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        size = root.find('size')
        
        width = int(size.find('width').text)
        height = int(size.find('height').text)
        
        filename = root.find('path').text.replace('\\','/')+'/'+root.find('filename').text
        count = 0
        for member in root.findall('object'):
            count = count + 1
            bndbox = member.find('bndbox')
            value = (filename,
                    width,
                    height,
                    member.find('name').text,
                    int(bndbox.find('xmin').text),
                    int(bndbox.find('ymin').text),
                    int(bndbox.find('xmax').text),
                    int(bndbox.find('ymax').text)
                    )
            xml_list.append(value)
        
        print('Processed {}. {} rows'.format(xml_file, count))
        
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax']
    return pd.DataFrame(xml_list, columns=column_name)

In [9]:
def create_tf_record(group,category_map):
    with tf.compat.v1.gfile.GFile(group.filename, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(category_map[row['class']]['id'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [10]:
def load_annotation(fname):
    tree = ET.parse(fname)
    root = tree.getroot()
    return root

In [11]:
def get_image_from_annotation(annotation: Element):
    return os.path.join(annotation.find('path').text, annotation.find('filename').text)

In [12]:
def run_inference_for_single_image(model, image):
  image = np.array(image)
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [13]:
def validated_list(original_list):
    new_list = []
    for annotation_path in original_list:
        try:
            annotation = load_annotation(annotation_path)
            img_path = get_image_from_annotation(annotation)
            img = Image.open(img_path)
            run_inference_for_single_image(MODEL, img)
            width, height = img.size
            num_objects = 0
            for member_object in annotation.findall('object'):
                num_objects += 1
                bnd_box = member_object.find('bndbox')
                xmin = int(bnd_box.find('xmin').text)
                ymin = int(bnd_box.find('ymin').text)
                xmax = int(bnd_box.find('xmax').text)
                ymax = int(bnd_box.find('ymax').text)
                if xmin>width or xmax>width or ymin>height or ymax>height:
                    raise Exception('Bounding box dimensions greater than max width or height')
            if num_objects > 0:
                new_list.append(annotation_path)
        except BaseException as e:
            print('Skipping {} because {}'.format(annotation_path, e))
    return new_list

In [14]:
def perform_partition(argv):    
    print(argv)
    split_percentage = 0.9
    annotation_names = [os.path.join(PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'],f) for f in os.listdir(PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'])]
    annotation_names = validated_list(annotation_names)
    shuffle(annotation_names)
    num_annotations = len(annotation_names)
    num_train = int(num_annotations*split_percentage)
    train_annotations, test_annotations = annotation_names[0:num_train],annotation_names[num_train:]

    if os.path.exists(PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH']):
        rmtree(PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH'])

    if os.path.exists(PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH']):
        rmtree(PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH'])

    def generate_record_and_csv(source_base: str, target_base: str, annotations):
        if not os.path.exists(target_base):
            os.makedirs(target_base)

        xml_df = xml_to_csv(annotations)
        xml_df.to_csv(os.path.join(target_base,'annotations_csv.csv'), index=None)
        print('Finished outputting {}'.format(os.path.join(target_base,'annotations_csv.csv')))
        
        # Finished outputting the CSV record above, now converting it to a record
        with tf.compat.v1.python_io.TFRecordWriter(os.path.join(target_base,'tensorflow_record.record')) as writer:
            
            data = namedtuple('data', ['filename', 'object'])
            gb = xml_df.groupby('filename')
            grouped = [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]
            
            category_map = create_name_index()
            for group in grouped:
                tf_record = create_tf_record(group, category_map)
                writer.write(tf_record.SerializeToString())


    generate_record_and_csv(
        PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'], 
        PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH'], 
        train_annotations)

    generate_record_and_csv(
        PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'], 
        PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH'], 
        test_annotations)
    
    print('Vini','Vidi','Vici')

In [15]:
tf.compat.v1.app.run(perform_partition)

['c:\\users\\letlhogonolo segoe\\pycharmprojects\\models\\my-goodness-experiments\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Letlhogonolo Segoe\\AppData\\Roaming\\jupyter\\runtime\\kernel-cf3cd2e2-9761-4c54-aad6-4debc3859a68.json']
Skipping D:/university/datasets/Potholes/newAnnotations\299.xml because  input must be 4-dimensional[1,576,1024]
	 [[node Preprocessor/map/while/ResizeImage/resize/ResizeBilinear (defined at <ipython-input-6-e9fbed0e00b0>:2) ]] [Op:__inference_pruned_6653]

Function call stack:
pruned

Skipping D:/university/datasets/Potholes/newAnnotations\407.xml because  PartialTensorShape: Incompatible shapes during merge: [300,300,4] vs. [300,300,3]
	 [[node Preprocessor/map/TensorArrayStack/TensorArrayGatherV3 (defined at <ipython-input-6-e9fbed0e00b0>:2) ]] [Op:__inference_pruned_6653]

Function call stack:
pruned

Skipping D:/university/datasets/Potholes/newAnnotations\478.xml because  input must be 4-dimensional[1,400,700]
	 [[node Preprocessor/m

I0610 20:52:32.315405 17100 label_map_util.py:120] Ignore item 0 since it falls outside of requested label range.
I0610 20:52:34.109851 17100 label_map_util.py:120] Ignore item 0 since it falls outside of requested label range.
c:\users\letlhogonolo segoe\pycharmprojects\models\my-goodness-experiments\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 